In [ ]:
import sys
sys.path.append("../")
from functools import partial

import torch
from torch.utils.data import DataLoader

import transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM

from src.models.rnn_model import RNN
from src.models.learned_skip_masking import DeepSkipAugmenter
from src.data.dataio import DataFiles, Dataset, truncate_fn

transformers.logging.set_verbosity_error()

In [ ]:
BERT_MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=BERT_MODEL_NAME)

data_files = DataFiles.from_dir(
    # "/home/pavlo/comp-550/comp-550-project/data/articles/clean_articles/training"
    "/Users/od/Documents/Mila/COMP-550/Final Project/comp-550-project/data/articles/clean_articles/training"
)

train_dataset = Dataset(data_files)
# train_dataset = train_dataset.map(
#     partial(
#         truncate_fn, 
#         tokenizer=tokenizer, 
#         max_seq_length=128, 
#         fill_to_max=False
#     )
# )


In [ ]:
# data_loader = DataLoader(train_dataset, batch_size=10)

model = DeepSkipAugmenter()

In [ ]:
model.train(train_dataset)